In [1]:
import json # import json module
import os
import os.path as pth
from glob import glob
import cv2
import matplotlib.image as img
import numpy as np
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline 
from PIL import Image 

print('ready')


ready


In [4]:
with open('./data/for_predict_color/list_description_inshop.json') as json_file:
    json_data = json.load(json_file)
    

color_list = []
for i in json_data:
    color_list.append(i['color'])
    
color_set = set(color_list)


print(len(color_list))
print(len(color_set))


id_list = []
for i in json_data:
    id_list.append(i['item'])
    
id_set = set(id_list)


print(len(id_list))
print(len(id_set))

print(json_data[3])

8081
805
8081
7982
{'item': 'id_00000004', 'color': 'Black-cream', 'description': ['Made to stun with an effortlessly ethereal, bohemian design, this open-shoulder top is crafted from crinkled gauze with long bell sleeves and a tasseled self-tie neckline. Those details would be enough to lend it a covetable look, but the contrast geo embroidery along its bodice is what makes it classically chic. ', 'Unlined', 'Lightweight, woven', '100% rayon', '24" full length, 43" chest, 45" waist, 26" sleeve length', 'Measured from Small', 'Hand wash cold', 'Made in China']}


In [5]:
df_cloth_bbox = pd.read_csv('./data/for_predict_color/list_bbox_inshop.txt', sep='\s+', skiprows=1)
display(df_cloth_bbox.head())

,image_name,clothes_type,pose_type,x_1,y_1,x_2,y_2
0,img/WOMEN/Blouses_Shirts/id_00000001/02_1_fron...,1,1,50,49,208,235
1,img/WOMEN/Blouses_Shirts/id_00000001/02_2_side...,1,2,119,48,136,234
2,img/WOMEN/Blouses_Shirts/id_00000001/02_3_back...,1,3,50,42,213,240
3,img/WOMEN/Blouses_Shirts/id_00000001/02_4_full...,1,4,82,30,162,129
4,img/WOMEN/Dresses/id_00000002/02_1_front.jpg,3,1,65,45,233,252


In [6]:

# 이미지 패스 아이디별로 묶음

path_color_classi = os.path.join('./data/for_predict_color/img')

tmp_list = []
img_path_sort_by_id = {}

for i in id_set:
    tmp_list = glob(os.path.join(path_color_classi, '*', '*', i, '*.jpg'))
    img_path_sort_by_id[i] = tmp_list

print(img_path_sort_by_id['id_00004151'])

['./data/for_predict_color/img/WOMEN/Blouses_Shirts/id_00004151/02_3_back.jpg', './data/for_predict_color/img/WOMEN/Blouses_Shirts/id_00004151/02_1_front.jpg', './data/for_predict_color/img/WOMEN/Blouses_Shirts/id_00004151/02_7_additional.jpg', './data/for_predict_color/img/WOMEN/Blouses_Shirts/id_00004151/02_2_side.jpg']


In [7]:
# 이미지의 id와 color로 이루어진 튜플로 리스트 만들기

id_color_list = []
for i in json_data:
    id_color_list.append((i['item'], i['color']))
    
print(id_color_list[0:10])

[('id_00000001', 'Cream'), ('id_00000002', 'Black-blush'), ('id_00000003', 'Heather grey'), ('id_00000004', 'Black-cream'), ('id_00000005', 'Burgundy'), ('id_00000006', 'Heather grey-black'), ('id_00000007', 'Heather grey-black'), ('id_00000008', 'Black'), ('id_00000009', 'Cream-multi'), ('id_00000010', 'Black')]


In [8]:
# 색깔별로 이미지패스 묶음

img_color_and_path = {}

for i in color_set:
    tmplist = []
    for j in id_color_list:
        if i == j[1]:
            paths = glob(os.path.join(path_color_classi, '*', '*', j[0], '*.jpg'))
            tmplist = tmplist + paths
        else:
            pass
    img_color_and_path[i] = tmplist

In [9]:
# 색깔별 이미지 개수가 100개 이하인 색들은 날려버림

# 43194개
# 98색

tmp = 0
n = 0 
del_list = []
for i in img_color_and_path:
    if len(img_color_and_path[i]) > 100:
        print(i, ' : ',len(img_color_and_path[i]))
        tmp += len(img_color_and_path[i])
        n += 1
    else:
        del_list.append(i)
    
print(' ')
print(' ')
print(tmp)
print(n)

for i in del_list:
    del img_color_and_path[i]

Taupe  :  691
Aqua  :  316
Cream-black  :  1756
Mauve  :  432
Wine  :  661
Lavender  :  185
Burgundy  :  2569
Heather grey-black  :  358
Rust  :  1016
Cream-blue  :  157
Blue-black  :  116
Black-cream  :  1659
Periwinkle  :  261
Grey  :  499
Black-white  :  953
Beige  :  225
Orange  :  117
White-navy  :  116
Navy-multi  :  155
Eggplant  :  115
Black  :  3440
Ivory  :  208
Hot pink  :  118
Blue-white  :  282
Navy  :  895
Black-black  :  193
Georgia peach  :  220
Blue-navy  :  116
Tangerine  :  112
Red-white  :  111
Oatmeal  :  605
Blue  :  226
Black-multi  :  286
Navy-white  :  404
Cream-multi  :  261
Navy-red  :  201
Amber  :  654
Teal  :  692
Lime  :  204
Pink  :  348
Peach  :  228
Dark grey  :  114
Brown  :  217
Rose  :  327
Blue-cream  :  417
Red-black  :  153
Bubble gum  :  187
Light denim  :  630
Taupe-cream  :  118
Navy-cream  :  365
Red  :  226
White-multi  :  153
Dusty pink  :  129
Cream-red  :  128
Purple  :  148
Black-pink  :  215
Heather grey  :  1576
Taupe-black  :  134
Red

In [11]:
# 색깔별로 카테고리 폴더 생성

def make_folder(folder_name): # 폴더 생성 함수

    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)


for i in img_color_and_path:
    make_folder('./data/train_by_color/' + i + '/')
    print(i)

Taupe
Aqua
Cream-black
Mauve
Wine
Lavender
Burgundy
Heather grey-black
Rust
Cream-blue
Blue-black
Black-cream
Periwinkle
Grey
Black-white
Beige
Orange
White-navy
Navy-multi
Eggplant
Black
Ivory
Hot pink
Blue-white
Navy
Black-black
Georgia peach
Blue-navy
Tangerine
Red-white
Oatmeal
Blue
Black-multi
Navy-white
Cream-multi
Navy-red
Amber
Teal
Lime
Pink
Peach
Dark grey
Brown
Rose
Blue-cream
Red-black
Bubble gum
Light denim
Taupe-cream
Navy-cream
Red
White-multi
Dusty pink
Cream-red
Purple
Black-pink
Heather grey
Taupe-black
Red-cream
Denim washed
Royal
Olive
Cream
Grey-black
Cream-grey
Yellow
Peacock
Black-gold
Dark denim
Tomato
Light blue
Dark navy
White
Mustard
Teal-black
Blue-multi
Seafoam
Black-blue
Black-grey
Coral
Blush
Denim
Charcoal
Oatmeal-black
Silver
Medium denim
Indigo
Khaki
Black-taupe
Heather grey-white
Light grey
Black-red
Ivory-black
Black-ivory
White-black
White-blue
Sunset gold
Cream-navy


In [13]:
def make_crop(file_path, my_bbox_df):
    img = Image.open(file_path)
    crop_square = (my_bbox_df['x_1'], my_bbox_df['y_1'], my_bbox_df['x_2'], my_bbox_df['y_2'])
    
    crop_img = img.crop(crop_square)
    
    return crop_img



new_data_path_color = './data/train_by_color/'


for i in img_color_and_path:
    m = 0
    for j in img_color_and_path[i]:
        # print(j)
        image = make_crop(j, df_cloth_bbox.loc[df_cloth_bbox['image_name'] == j[25:]])
        img_resize = image.resize((299,299))
        img_resize.save(new_data_path_color + i + '/{}.jpg'.format(m))
        m += 1
    print(i, ' complete')


Taupe  complete
Aqua  complete
Cream-black  complete
Mauve  complete
Wine  complete
Lavender  complete
Burgundy  complete
Heather grey-black  complete
Rust  complete
Cream-blue  complete
Blue-black  complete
Black-cream  complete
Periwinkle  complete
Grey  complete
Black-white  complete
Beige  complete
Orange  complete
White-navy  complete
Navy-multi  complete
Eggplant  complete
Black  complete
Ivory  complete
Hot pink  complete
Blue-white  complete
Navy  complete
Black-black  complete
Georgia peach  complete
Blue-navy  complete
Tangerine  complete
Red-white  complete
Oatmeal  complete
Blue  complete
Black-multi  complete
Navy-white  complete
Cream-multi  complete
Navy-red  complete
Amber  complete
Teal  complete
Lime  complete
Pink  complete
Peach  complete
Dark grey  complete
Brown  complete
Rose  complete
Blue-cream  complete
Red-black  complete
Bubble gum  complete
Light denim  complete
Taupe-cream  complete
Navy-cream  complete
Red  complete
White-multi  complete
Dusty pink  compl